# Preprocessing of wind turbine data

Cells in this notebook can be run independently provided the first two ("Imports" and "Preprocessing function...") are run first.

1. [Imports.](#Cell1)
1. [Preprocessing function, good for training data as well as testing data.](#Cell2)
1. [Preprocess the training data set, output a pickle file.](#Cell3)
1. [Preprocess the training data set in Feather format for notebooks running R kernel.](#Cell4)
1. [Preprocess the testing data set, output a pickle file.](#Cell5)
1. [Preprocess the testing data set in Feather format for notebooks running R kernel.](#Cell6)
1. [Preprocess the solution file (the testing data set with TARGETVAR included). Output in pickle and Feather.](#Cell7)

<a id='Cell1'></a>

In [3]:
'''
Imports.
'''
import feather
import pandas as pd
import numpy as np
from time import strftime
import datetime
from IPython.display import display_html

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

<a id='Cell2'></a>

In [2]:
'''
Preprocessing function, good for training data as well as testing data.
'''
def preProcess(fname, Nroll=3, inclusive=False, keepID=False):
    '''
    Read in the wind turbine data, store in a Pandas dataframe, and rearrange.

    1.  Records in the input csv file are assumed to have the following form:
            ID, ZONEID, TIMESTAMP, TARGETVAR, U10, V10, U100, V100
        where:
            ID =         Unique ID for each observation
            ZONEID =     Zone/Turbine ID, a number between 1 and 10
            TIMESTAMP =  Time of observation, in the format "YYYYMMDD h:mm" or "YYYYMMDD hh:mm"
            TARGETVAR =  Wind turbine output (only present in the training data set)
            U10 =        Zonal Wind Vector at 10 m
            V10 =        Meridional Wind Vector at 10 m
            U100 =       Zonal Wind vector at 100 m
            V100 =       Meridional Wind vector at 100 m
    
    2. If Nroll<=1, column names in the output data frame have the form:
            TARGETVAR U10 V10 U100 V100 YEAR DAYOFYEAR HOUR
       and TARGETVAR, U10, V10, U100, V100 each have a subindex 1 ... 10 indicating zone id.
       
       If Nroll>1, the structure is the same, but the U10, V10, U100, V100 columns are replaced
       by rolling means over Nroll measurements, and the names are U10_rmX, V10_rmX, U100_rmX, 
       and V100_rmX, where X equals Nroll.
       
       If Nroll>1 and inclusive==True, all rolling means from 1 to Nroll are kept in the 
       dataframe. If inclusive==False, only the rolling mean corresponding to Nroll is kept.
    '''

    # Read in the windmill data, parsing the third column as datetimes. Make a hierarchical
    # index out of the timestamp and zone id columns.  Then drop the measurement ID column.
    df0 = pd.read_csv(fname, header=0, parse_dates=[2], index_col=[2,1])
    if not keepID: 
        df0 = df0.drop('ID', 1)
        print('ID column dropped')
    else:
        print('ID column kept')
        
    # Unstack the inner level of the index. The index is thus reduced to the timestamp, and 
    # the zone ids become subcolumns within the existing columns. In other words, where in
    # df0 there are 10 rows per time stamp, one for each zone id, in df1 there is only 
    # 1 row, which contains the measurements made at the given time stamp in all the zone ids.
    df1 = df0.unstack()

    # Put the timestamp index back as a dataframe column and verify that all delta-times are one hour.
    df2              = df1.reset_index()
    dtimes           = df2["TIMESTAMP"].subtract(df2["TIMESTAMP"].shift(+1))
    Ntimepoints      = len(df2)
    Ndeltatimes1h    = dtimes[dtimes==np.timedelta64(1, 'h')].count()
    assert (Ntimepoints == Ndeltatimes1h+1), 'Delta times not all equal to 1 h!'
    print('Number of time points:                 {0}'.format(Ntimepoints))
    print('Number of delta-times equal to 1 hour: {0}'.format(Ndeltatimes1h))
    
    # Remove duplicate wind measurements (5 is duplicate with 4, and 8 with 7)
    nturbines         = 10
    wind_measurements = [1, 2, 3, 4, 6, 7, 9, 10]
    for i in range(1,nturbines+1):
        if i not in wind_measurements:
            df2 = df2.drop([("U10",i), ("V10",i), ("U100",i), ("V100",i)], axis=1)
    
    # Add rolling means of the wind measurements
    if Nroll>1:
        NrollSet = range(2, Nroll+1) if inclusive else [Nroll]
        for n in NrollSet:
            print('Computing rolling means over {0} measurements'.format(n))
            dname = "_rm"+str(n)
            for i in wind_measurements:
                df2[("U10"+dname,i)]  = df2[("U10",i)].rolling(n, min_periods=1).mean()
                df2[("U100"+dname,i)] = df2[("U100",i)].rolling(n, min_periods=1).mean()
                df2[("V10"+dname,i)]  = df2[("V10",i)].rolling(n, min_periods=1).mean()
                df2[("V100"+dname,i)] = df2[("V100",i)].rolling(n, min_periods=1).mean()
        if inclusive:
            df2.rename(columns={"U10": "U10_rm1", "U100": "U100_rm1", "V10": "V10_rm1", "V100": "V100_rm1"}, inplace=True)
        else:
            df2 = df2.drop(["U10", "U100", "V10", "V100"], axis=1, level=0)
    else:
        print('No rolling means computed')
    
    # Extract year, day of year, and hour from timestamp, then drop timestamp column
    df2["YEAR"]      = df2["TIMESTAMP"].map(lambda x: x.year)
    df2["DAYOFYEAR"] = df2["TIMESTAMP"].map(lambda x: x.timetuple().tm_yday)
    df2["HOUR"]      = df2["TIMESTAMP"].map(lambda x: x.hour)
    df2              = df2.drop('TIMESTAMP', axis=1, level=0)

    return df2

<a id='Cell3'></a>

In [5]:
'''
Preprocess the training data set, output a pickle or csv file.
'''
output_format = "csv"

# Load and preprocess data
train_csv = "data/Train_O4UPEyW.csv"
train_df  = preProcess(train_csv, Nroll=3, inclusive=False, keepID=False)
display_html(train_df.head())
print("Data frame dimensions: {0}".format(train_df.shape))

# Output dataframe
if output_format == "pkl":
    train_fname = "data/Train_pp_" + strftime("%Y_%m_%d_%H_%M_%S") + ".pkl"
    try:
        train_df.to_pickle(train_fname)
        print('\nTraining data frame saved to {0}'.format(train_fname))
    except:
        print('\nError saving training data frame to {0}'.format(train_fname))
elif output_format == "csv":
    train_fname = "data/Train_pp_" + strftime("%Y_%m_%d_%H_%M_%S") + ".csv"
    try:
        train_df.to_csv(train_fname, header=True, index=False)
        print('\nTraining data frame saved to {0}'.format(train_fname))
    except:
        print('\nError saving training data frame to {0}'.format(train_fname))
else:
    print('\nUnrecognized output format')

ID column dropped
Number of time points:                 13871
Number of delta-times equal to 1 hour: 13870
Computing rolling means over 3 measurements


TARGETVAR 
 U10_rm3 
 U100_rm3 
 V10_rm3 
 V100_rm3 
 U10_rm3 
 U100_rm3 
 V10_rm3 
 V100_rm3 
 U10_rm3 
 U100_rm3 
 V10_rm3 
 V100_rm3 
 U10_rm3 
 U100_rm3 
 V10_rm3 
 V100_rm3 
 U10_rm3 
 U100_rm3 
 V10_rm3 
 V100_rm3 
 U10_rm3 
 U100_rm3 
 V10_rm3 
 V100_rm3 
 U10_rm3 
 U100_rm3 
 V10_rm3 
 V100_rm3 
 U10_rm3 
 U100_rm3 
 V10_rm3 
 V100_rm3 
 YEAR 
 DAYOFYEAR 
 HOUR 
 
 
 ZONEID 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 1 
 1 
 1 
 1 
 2 
 2 
 2 
 2 
 3 
 3 
 3 
 3 
 4 
 4 
 4 
 4 
 6 
 6 
 6 
 6 
 7 
 7 
 7 
 7 
 9 
 9 
 9 
 9 
 10 
 10 
 10 
 10 
 
 
 
 
 
 
 
 0 
 0.000000 
 0.596273 
 0.425465 
 0.378229 
 0.273678 
 0.268101 
 0.000000 
 0.000000 
 0.0 
 0.594361 
 2.124600 
 2.864280 
 -2.681966 
 -3.666076 
 -0.171642 
 -0.112594 
 -5.466031 
 -7.101347 
 2.193217 
 3.028435 
 -4.649168 
 -6.221648 
 0.534894 
 0.765355 
 -3.660243 
 -4.486657 
 1.205572 
 1.467827 
 -2.417913 
 -2.956134 
 -0.307548 
 -0.625438 
 0.371263 
 0.298533 
 1.903254 
 2.516723 
 -1.495002 
 -2.078063 
 1.655790 
 2.024257 
 -4.649593 
 -5.992232 
 2012 
 1 
 1 
 
 
 1 
 0.054879 
 0.411180 
 0.363851 
 0.063012 
 0.086796 
 0.034542 
 0.014708 
 0.014954 
 0.0 
 0.569679 
 2.323147 
 3.104569 
 -2.239463 
 -3.065419 
 -0.130250 
 -0.063347 
 -5.054618 
 -6.499062 
 2.581679 
 3.504769 
 -4.296741 
 -5.737868 
 0.432854 
 0.617615 
 -3.168337 
 -3.850205 
 1.093350 
 1.308483 
 -1.997769 
 -2.446593 
 -0.083768 
 -0.300240 
 0.561424 
 0.548015 
 2.038275 
 2.676916 
 -1.069321 
 -1.542488 
 1.917199 
 2.323699 
 -4.417269 
 -5.655159 
 2012 
 1 
 2 
 
 
 2 
 0.110234 
 0.167243 
 0.297129 
 0.036704 
 0.006811 
 0.020621 
 0.035098 
 0.071785 
 0.0 
 0.330539 
 2.439502 
 3.239195 
 -1.767148 
 -2.448310 
 -0.168838 
 -0.117311 
 -4.574210 
 -5.829165 
 2.800738 
 3.788456 
 -3.912656 
 -5.213050 
 0.266623 
 0.404779 
 -2.788582 
 -3.372843 
 0.933317 
 1.104798 
 -1.668908 
 -2.051307 
 0.169526 
 0.024769 
 0.865015 
 0.907596 
 2.129765 
 2.795976 
 -0.667349 
 -1.033489 
 2.021010 
 2.430872 
 -4.025660 
 -5.120850 
 2012 
 1 
 3 
 
 
 3 
 0.165116 
 0.037326 
 0.235438 
 0.034342 
 0.018646 
 0.001867 
 0.045055 
 0.066796 
 0.0 
 0.211308 
 2.550470 
 3.356180 
 -0.921040 
 -1.344800 
 -0.338537 
 -0.337128 
 -3.725405 
 -4.661657 
 3.156662 
 4.216958 
 -3.105872 
 -4.110987 
 -0.051525 
 0.001372 
 -2.168197 
 -2.593842 
 0.612365 
 0.702900 
 -1.111840 
 -1.384754 
 0.574441 
 0.545537 
 1.494201 
 1.683343 
 2.234535 
 2.937678 
 0.096703 
 -0.066291 
 2.161491 
 2.567102 
 -3.257490 
 -4.094716 
 2012 
 1 
 4 
 
 
 4 
 0.156940 
 0.051206 
 0.120904 
 0.033554 
 0.034812 
 0.015174 
 0.050160 
 0.034797 
 0.0 
 0.172140 
 2.458537 
 3.227120 
 -0.192194 
 -0.412313 
 -0.729286 
 -0.828273 
 -3.050932 
 -3.777620 
 3.146628 
 4.201108 
 -2.471149 
 -3.256237 
 -0.413204 
 -0.434678 
 -1.829896 
 -2.186534 
 0.239176 
 0.264020 
 -0.753096 
 -0.969672 
 0.774243 
 0.777431 
 2.212967 
 2.584736 
 2.143636 
 2.845034 
 0.739802 
 0.759714 
 2.040194 
 2.392003 
 -2.388384 
 -2.980721 
 2012 
 1 
 5

Data frame dimensions: (13871, 45)

Training data frame saved to data/Train_pp_2018_10_21_12_22_36.csv


<a id='Cell4'></a>

In [31]:
'''
Preprocess the training data set in Feather format for notebooks running R kernel.
'''
# Load and preprocess data
train_csv = "data/Train_O4UPEyW.csv"
train_df  = preProcess(train_csv, Nroll=4, inclusive=True, keepID=False)

# Rename dataframe columns
predictors  = [item[0]+'_'+str(item[1]) for item in train_df.columns.values if item[0]!='TARGETVAR']
predictors  = [predictor if predictor[-1]!='_' else predictor[:-1] for predictor in predictors]
targets     = [item[0]+'_'+str(item[1]) for item in train_df.columns.values if item[0]=='TARGETVAR']
train_df.columns = targets+predictors
display_html(train_df.head())
print("Data frame dimensions: {0}".format(train_df.shape))

# Output data frame in feather format
train_fname = "data/Train_pp_" + strftime("%Y_%m_%d_%H_%M_%S") + ".feather"
try:
    feather.write_dataframe(train_df, train_fname)
    print('\nTraining data frame saved to {0}'.format(train_fname))
except:
    print('\nError saving training data frame to {0}'.format(train_fname))

ID column dropped
Number of time points:                 13871
Number of delta-times equal to 1 hour: 13870
Computing rolling means over 2 measurements
Computing rolling means over 3 measurements
Computing rolling means over 4 measurements


,TARGETVAR_1,TARGETVAR_2,TARGETVAR_3,TARGETVAR_4,TARGETVAR_5,TARGETVAR_6,TARGETVAR_7,TARGETVAR_8,TARGETVAR_9,TARGETVAR_10,U10_rm1_1,U10_rm1_2,U10_rm1_3,U10_rm1_4,U10_rm1_6,U10_rm1_7,U10_rm1_9,U10_rm1_10,V10_rm1_1,V10_rm1_2,V10_rm1_3,V10_rm1_4,V10_rm1_6,V10_rm1_7,V10_rm1_9,V10_rm1_10,U100_rm1_1,U100_rm1_2,U100_rm1_3,U100_rm1_4,U100_rm1_6,U100_rm1_7,U100_rm1_9,U100_rm1_10,V100_rm1_1,V100_rm1_2,V100_rm1_3,V100_rm1_4,V100_rm1_6,V100_rm1_7,V100_rm1_9,V100_rm1_10,U10_rm2_1,U100_rm2_1,V10_rm2_1,V100_rm2_1,U10_rm2_2,U100_rm2_2,V10_rm2_2,V100_rm2_2,U10_rm2_3,U100_rm2_3,V10_rm2_3,V100_rm2_3,U10_rm2_4,U100_rm2_4,V10_rm2_4,V100_rm2_4,U10_rm2_6,U100_rm2_6,V10_rm2_6,V100_rm2_6,U10_rm2_7,U100_rm2_7,V10_rm2_7,V100_rm2_7,U10_rm2_9,U100_rm2_9,V10_rm2_9,V100_rm2_9,U10_rm2_10,U100_rm2_10,V10_rm2_10,V100_rm2_10,U10_rm3_1,U100_rm3_1,V10_rm3_1,V100_rm3_1,U10_rm3_2,U100_rm3_2,V10_rm3_2,V100_rm3_2,U10_rm3_3,U100_rm3_3,V10_rm3_3,V100_rm3_3,U10_rm3_4,U100_rm3_4,V10_rm3_4,V100_rm3_4,U10_rm3_6,U100_rm3_6,V10_rm3_6,V100_rm3_6,U10_rm3_7,U100_rm3_7,V10_rm3_7,V100_rm3_7,U10_rm3_9,U100_rm3_9,V10_rm3_9,V100_rm3_9,U10_rm3_10,U100_rm3_10,V10_rm3_10,V100_rm3_10,U10_rm4_1,U100_rm4_1,V10_rm4_1,V100_rm4_1,U10_rm4_2,U100_rm4_2,V10_rm4_2,V100_rm4_2,U10_rm4_3,U100_rm4_3,V10_rm4_3,V100_rm4_3,U10_rm4_4,U100_rm4_4,V10_rm4_4,V100_rm4_4,U10_rm4_6,U100_rm4_6,V10_rm4_6,V100_rm4_6,U10_rm4_7,U100_rm4_7,V10_rm4_7,V100_rm4_7,U10_rm4_9,U100_rm4_9,V10_rm4_9,V100_rm4_9,U10_rm4_10,U100_rm4_10,V10_rm4_10,V100_rm4_10,YEAR,DAYOFYEAR,HOUR
0,0.000000,0.596273,0.425465,0.378229,0.273678,0.268101,0.000000,0.000000,0.0,0.594361,2.124600,-0.171642,2.193217,0.534894,1.205572,-0.307548,1.903254,1.655790,-2.681966,-5.466031,-4.649168,-3.660243,-2.417913,0.371263,-1.495002,-4.649593,2.864280,-0.112594,3.028435,0.765355,1.467827,-0.625438,2.516723,2.024257,-3.666076,-7.101347,-6.221648,-4.486657,-2.956134,0.298533,-2.078063,-5.992232,2.124600,2.864280,-2.681966,-3.666076,-0.171642,-0.112594,-5.466031,-7.101347,2.193217,3.028435,-4.649168,-6.221648,0.534894,0.765355,-3.660243,-4.486657,1.205572,1.467827,-2.417913,-2.956134,-0.307548,-0.625438,0.371263,0.298533,1.903254,2.516723,-1.495002,-2.078063,1.655790,2.024257,-4.649593,-5.992232,2.124600,2.864280,-2.681966,-3.666076,-0.171642,-0.112594,-5.466031,-7.101347,2.193217,3.028435,-4.649168,-6.221648,0.534894,0.765355,-3.660243,-4.486657,1.205572,1.467827,-2.417913,-2.956134,-0.307548,-0.625438,0.371263,0.298533,1.903254,2.516723,-1.495002,-2.078063,1.655790,2.024257,-4.649593,-5.992232,2.124600,2.864280,-2.681966,-3.666076,-0.171642,-0.112594,-5.466031,-7.101347,2.193217,3.028435,-4.649168,-6.221648,0.534894,0.765355,-3.660243,-4.486657,1.205572,1.467827,-2.417913,-2.956134,-0.307548,-0.625438,0.371263,0.298533,1.903254,2.516723,-1.495002,-2.078063,1.655790,2.024257,-4.649593,-5.992232,2012,1,1
1,0.054879,0.411180,0.363851,0.063012,0.086796,0.034542,0.014708,0.014954,0.0,0.569679,2.521695,-0.088859,2.970141,0.330813,0.981127,0.140013,2.173296,2.178609,-1.796960,-4.643205,-3.944314,-2.676430,-1.577625,0.751585,-0.643641,-4.184944,3.344859,-0.014100,3.981102,0.469875,1.149139,0.024957,2.837109,2.623141,-2.464761,-5.896778,-5.254089,-3.213752,-1.937051,0.797498,-1.006914,-5.318086,2.323147,3.104569,-2.239463,-3.065419,-0.130250,-0.063347,-5.054618,-6.499062,2.581679,3.504769,-4.296741,-5.737868,0.432854,0.617615,-3.168337,-3.850205,1.093350,1.308483,-1.997769,-2.446593,-0.083768,-0.300240,0.561424,0.548015,2.038275,2.676916,-1.069321,-1.542488,1.917199,2.323699,-4.417269,-5.655159,2.323147,3.104569,-2.239463,-3.065419,-0.130250,-0.063347,-5.054618,-6.499062,2.581679,3.504769,-4.296741,-5.737868,0.432854,0.617615,-3.168337,-3.850205,1.093350,1.308483,-1.997769,-2.446593,-0.083768,-0.300240,0.561424,0.548015,2.038275,2.676916,-1.069321,-1.542488,1.917199,2.323699,-4.417269,-5.655159,2.323147,3.104569,-2.239463,-3.065419,-0.130250,-0.063347,-5.054618,-6.499062,2.581679,3.504769,-4.296741,-5.737868,0.432854,0.617615,-3.168337,-3.850205,1.093350,1.308483,-1.

Data frame dimensions: (13871, 141)

Training data frame saved to data/Train_pp_2017_11_04_21_59_47.feather


<a id='Cell5'></a>

In [11]:
'''
Preprocess the testing data set, output a pickle file.
'''
test_csv  = "data/Test_uP7dymh.csv"

test_df = preProcess(test_csv, Nroll=3, inclusive=False, keepID=True)
display_html(test_df.head())
print("Data frame dimensions: {0}".format(test_df.shape))

test_fname = "data/Test_pp_" + strftime("%Y_%m_%d_%H_%M_%S") + ".pkl"
try:
    test_df.to_pickle(test_fname)
    print('\nTesting data frame saved to {0}'.format(test_fname))
except:
    print('\nError saving testing data frame to {0}'.format(test_fname))

ID column kept
Number of time points:                 2929
Number of delta-times equal to 1 hour: 2928
Computing rolling means over 3 measurements

Dataframe test_df:
              ID                                                              \
ZONEID         1         2         3         4         5         6         7   
0       30010001  30010002  30010003  30010004  30010005  30010006  30010007   
1       30010101  30010102  30010103  30010104  30010105  30010106  30010107   
2       30010201  30010202  30010203  30010204  30010205  30010206  30010207   
3       30010301  30010302  30010303  30010304  30010305  30010306  30010307   
4       30010401  30010402  30010403  30010404  30010405  30010406  30010407   

                                     ...   U100_rm3   V10_rm3  V100_rm3  \
ZONEID         8         9        10 ...          9         9         9   
0       30010008  30010009  30010010 ...   1.473541 -3.757118 -5.748985   
1       30010108  30010109  30010110 ...   2.42

<a id='Cell6'></a>

In [32]:
'''
Preprocess the testing data set in Feather format for notebooks running R kernel.
'''
test_csv  = "data/Test_uP7dymh.csv"

test_df = preProcess(test_csv, Nroll=4, inclusive=True, keepID=True)

# Rename dataframe columns
predictors  = [item[0]+'_'+str(item[1]) for item in test_df.columns.values]
predictors  = [predictor if predictor[-1]!='_' else predictor[:-1] for predictor in predictors]
test_df.columns = predictors
display_html(test_df.head())
print("Data frame dimensions: {0}".format(test_df.shape))
    
# Output data frame in feather format
test_fname = "data/Test_pp_" + strftime("%Y_%m_%d_%H_%M_%S") + ".feather"
try:
    feather.write_dataframe(test_df, test_fname)
    print('\nTesting data frame saved to {0}'.format(test_fname))
except:
    print('\nError saving testing data frame to {0}'.format(test_fname))

ID column kept
Number of time points:                 2929
Number of delta-times equal to 1 hour: 2928
Computing rolling means over 2 measurements
Computing rolling means over 3 measurements
Computing rolling means over 4 measurements


,ID_1,ID_2,ID_3,ID_4,ID_5,ID_6,ID_7,ID_8,ID_9,ID_10,U10_rm1_1,U10_rm1_2,U10_rm1_3,U10_rm1_4,U10_rm1_6,U10_rm1_7,U10_rm1_9,U10_rm1_10,V10_rm1_1,V10_rm1_2,V10_rm1_3,V10_rm1_4,V10_rm1_6,V10_rm1_7,V10_rm1_9,V10_rm1_10,U100_rm1_1,U100_rm1_2,U100_rm1_3,U100_rm1_4,U100_rm1_6,U100_rm1_7,U100_rm1_9,U100_rm1_10,V100_rm1_1,V100_rm1_2,V100_rm1_3,V100_rm1_4,V100_rm1_6,V100_rm1_7,V100_rm1_9,V100_rm1_10,U10_rm2_1,U100_rm2_1,V10_rm2_1,V100_rm2_1,U10_rm2_2,U100_rm2_2,V10_rm2_2,V100_rm2_2,U10_rm2_3,U100_rm2_3,V10_rm2_3,V100_rm2_3,U10_rm2_4,U100_rm2_4,V10_rm2_4,V100_rm2_4,U10_rm2_6,U100_rm2_6,V10_rm2_6,V100_rm2_6,U10_rm2_7,U100_rm2_7,V10_rm2_7,V100_rm2_7,U10_rm2_9,U100_rm2_9,V10_rm2_9,V100_rm2_9,U10_rm2_10,U100_rm2_10,V10_rm2_10,V100_rm2_10,U10_rm3_1,U100_rm3_1,V10_rm3_1,V100_rm3_1,U10_rm3_2,U100_rm3_2,V10_rm3_2,V100_rm3_2,U10_rm3_3,U100_rm3_3,V10_rm3_3,V100_rm3_3,U10_rm3_4,U100_rm3_4,V10_rm3_4,V100_rm3_4,U10_rm3_6,U100_rm3_6,V10_rm3_6,V100_rm3_6,U10_rm3_7,U100_rm3_7,V10_rm3_7,V100_rm3_7,U10_rm3_9,U100_rm3_9,V10_rm3_9,V100_rm3_9,U10_rm3_10,U100_rm3_10,V10_rm3_10,V100_rm3_10,U10_rm4_1,U100_rm4_1,V10_rm4_1,V100_rm4_1,U10_rm4_2,U100_rm4_2,V10_rm4_2,V100_rm4_2,U10_rm4_3,U100_rm4_3,V10_rm4_3,V100_rm4_3,U10_rm4_4,U100_rm4_4,V10_rm4_4,V100_rm4_4,U10_rm4_6,U100_rm4_6,V10_rm4_6,V100_rm4_6,U10_rm4_7,U100_rm4_7,V10_rm4_7,V100_rm4_7,U10_rm4_9,U100_rm4_9,V10_rm4_9,V100_rm4_9,U10_rm4_10,U100_rm4_10,V10_rm4_10,V100_rm4_10,YEAR,DAYOFYEAR,HOUR
0,30010001,30010002,30010003,30010004,30010005,30010006,30010007,30010008,30010009,30010010,0.566472,1.552273,-0.056684,0.691553,1.167131,-0.465222,0.781539,-0.165117,-3.866858,-3.037934,-3.197154,-1.882241,-2.284945,-4.393509,-3.757118,-2.658255,0.935726,2.819186,0.151737,1.262296,1.829496,-0.484770,1.473541,0.320843,-5.888026,-5.077052,-4.559475,-2.754384,-3.287274,-6.763927,-5.748985,-4.277716,0.566472,0.935726,-3.866858,-5.888026,1.552273,2.819186,-3.037934,-5.077052,-0.056684,0.151737,-3.197154,-4.559475,0.691553,1.262296,-1.882241,-2.754384,1.167131,1.829496,-2.284945,-3.287274,-0.465222,-0.484770,-4.393509,-6.763927,0.781539,1.473541,-3.757118,-5.748985,-0.165117,0.320843,-2.658255,-4.277716,0.566472,0.935726,-3.866858,-5.888026,1.552273,2.819186,-3.037934,-5.077052,-0.056684,0.151737,-3.197154,-4.559475,0.691553,1.262296,-1.882241,-2.754384,1.167131,1.829496,-2.284945,-3.287274,-0.465222,-0.484770,-4.393509,-6.763927,0.781539,1.473541,-3.757118,-5.748985,-0.165117,0.320843,-2.658255,-4.277716,0.566472,0.935726,-3.866858,-5.888026,1.552273,2.819186,-3.037934,-5.077052,-0.056684,0.151737,-3.197154,-4.559475,0.691553,1.262296,-1.882241,-2.754384,1.167131,1.829496,-2.284945,-3.287274,-0.465222,-0.484770,-4.393509,-6.763927,0.781539,1.473541,-3.757118,-5.748985,-0.165117,0.320843,-2.658255,-4.277716,2013,213,0
1,30010101,30010102,30010103,30010104,30010105,30010106,30010107,30010108,30010109,30010110,2.513531,1.856405,1.865527,2.099645,2.254329,-0.144996,2.461843,-0.349089,-4.475081,-3.909570,-3.798061,-1.608958,-1.636207,-5.039754,-4.089410,-3.789677,3.426426,2.678273,2.653962,2.709924,2.918636,-0.033150,3.378262,-0.350576,-6.415826,-5.350381,-5.098012,-2.039563,-2.081795,-7.279274,-5.847995,-5.323414,1.540001,2.181076,-4.170969,-6.151926,1.704339,2.748730,-3.473752,-5.213717,0.904422,1.402849,-3.497607,-4.828743,1.395599,1.986110,-1.745600,-2.396974,1.710730,2.374066,-1.960576,-2.684534,-0.305109,-0.258960,-4.716631,-7.021601,1.621691,2.425901,-3.923264,-5.798490,-0.257103,-0.014866,-3.223966,-4.800565,1.540001,2.181076,-4.170969,-6.151926,1.704339,2.748730,-3.473752,-5.213717,0.904422,1.402849,-3.497607,-4.828743,1.395599,1.986110,-1.745600,-2.396974,1.710730,2.374066,-1.960576,-2.684534,-0.305109,-0.258960,-4.716631,-7.021601,1.621691,2.425901,-3.923264,-5.798490,-0.257103,-0.014866,-3.223966,-4.800565,1.540001,2.181076,-4.170969,-6.151926,1.704339,2.748730,-3.473752,-5.213717,0.904422,1.402849,-3.497607,-4.828743,1.395599,1.986110,-1.745600,-2.396974,1.710730,2.374066,-1.960576,-2.684534,-0.305109,-0.258960,-4.7166

Data frame dimensions: (2929, 141)

Testing data frame saved to data/Test_pp_2017_11_04_22_01_57.feather


<a id='Cell7'></a>

In [5]:
'''
Preprocess the solution file. This is the testing data set with TARGETVAR included.
Preprocess it in the same way as the training data set, then output it in pickle and feather formats.
'''
# Load and preprocess data
solution_csv = "data/Solution.csv"
solution_df  = preProcess(solution_csv, Nroll=1, inclusive=False, keepID=False)
print('\nDataframe solution_df for pickle:\n')
display_html(solution_df.head())

# Output dataframe in pickle format
fname          = strftime("%Y_%m_%d_%H_%M_%S")
solution_fname = "data/Solution_pp_" + fname + ".pkl"
try:
    solution_df.to_pickle(solution_fname)
    print('\nSolution data frame saved to {0}'.format(solution_fname))
except:
    print('\nError saving solution data frame to {0}'.format(solution_fname))
    

# Rename dataframe columns for easier handling in R
fields  = [item[0]+'_'+str(item[1]) for item in solution_df.columns.values]
fields  = [field if field[-1]!='_' else field[:-1] for field in fields]
solution_df.columns = fields
print('\nDataframe solution_df for feather:\n')
display_html(solution_df.head())
    
# Output data frame in feather format
solution_fname = "data/Solution_pp_" + fname + ".feather"
try:
    feather.write_dataframe(solution_df, solution_fname)
    print('\nSolution data frame saved to {0}'.format(solution_fname))
except:
    print('\nError saving solution data frame to {0}'.format(solution_fname))

ID column dropped
Number of time points:                 2929
Number of delta-times equal to 1 hour: 2928
No rolling means computed

Dataframe solution_df for pickle:



TARGETVAR 
 U10 
 V10 
 U100 
 V100 
 YEAR 
 DAYOFYEAR 
 HOUR 
 
 
 ZONEID 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 
 
 
 
 
 
 
 0 
 0.000000 
 0.270840 
 0.070839 
 0.032609 
 0.029816 
 0.052044 
 0.140550 
 0.224602 
 0.054787 
 0.121128 
 0.566472 
 1.552273 
 -0.056684 
 0.691553 
 0.691553 
 1.167131 
 -0.465222 
 -0.465222 
 0.781539 
 -0.165117 
 -3.866858 
 -3.037934 
 -3.197154 
 -1.882241 
 -1.882241 
 -2.284945 
 -4.393509 
 -4.393509 
 -3.757118 
 -2.658255 
 0.935726 
 2.819186 
 0.151737 
 1.262296 
 1.262296 
 1.829496 
 -0.484770 
 -0.484770 
 1.473541 
 0.320843 
 -5.888026 
 -5.077052 
 -4.559475 
 -2.754384 
 -2.754384 
 -3.287274 
 -6.763927 
 -6.763927 
 -5.748985 
 -4.277716 
 2013 
 213 
 0 
 
 
 1 
 0.019168 
 0.195737 
 0.081343 
 0.028040 
 0.041659 
 0.051252 
 0.152884 
 0.150140 
 0.184262 
 0.235245 
 2.513531 
 1.856405 
 1.865527 
 2.099645 
 2.099645 
 2.254329 
 -0.144996 
 -0.144996 
 2.461843 
 -0.349089 
 -4.475081 
 -3.909570 
 -3.798061 
 -1.608958 
 -1.608958 
 -1.636207 
 -5.039754 
 -5.039754 
 -4.089410 
 -3.789677 
 3.426426 
 2.678273 
 2.653962 
 2.709924 
 2.709924 
 2.918636 
 -0.033150 
 -0.033150 
 3.378262 
 -0.350576 
 -6.415826 
 -5.350381 
 -5.098012 
 -2.039563 
 -2.039563 
 -2.081795 
 -7.279274 
 -7.279274 
 -5.847995 
 -5.323414 
 2013 
 213 
 1 
 
 
 2 
 0.286627 
 0.207469 
 0.107185 
 0.064430 
 0.052353 
 0.086763 
 0.199029 
 0.151965 
 0.389327 
 0.271003 
 2.730166 
 2.283594 
 1.522712 
 2.638951 
 2.638951 
 2.653013 
 0.056817 
 0.056817 
 2.637051 
 0.769431 
 -4.752177 
 -4.232779 
 -3.935561 
 -1.645848 
 -1.645848 
 -1.640399 
 -5.192765 
 -5.192765 
 -4.283922 
 -4.633122 
 3.915867 
 3.169549 
 2.140667 
 3.317253 
 3.317253 
 3.320464 
 0.207672 
 0.207672 
 3.845685 
 1.020502 
 -6.847804 
 -5.587994 
 -5.242004 
 -2.112832 
 -2.112832 
 -2.149975 
 -7.343893 
 -7.343893 
 -6.058642 
 -6.176686 
 2013 
 213 
 2 
 
 
 3 
 0.287943 
 0.395594 
 0.157022 
 0.055293 
 0.052712 
 0.103719 
 0.281050 
 0.217666 
 0.503941 
 0.272300 
 2.461083 
 3.095404 
 1.449740 
 2.958202 
 2.958202 
 2.892832 
 0.359725 
 0.359725 
 2.187819 
 1.873888 
 -4.583656 
 -4.070965 
 -4.013953 
 -1.850422 
 -1.850422 
 -1.892762 
 -5.167193 
 -5.167193 
 -4.210980 
 -5.047719 
 3.541562 
 4.191092 
 2.054430 
 3.704862 
 3.704862 
 3.611744 
 0.637481 
 0.637481 
 3.203953 
 2.476900 
 -6.545063 
 -5.364627 
 -5.389050 
 -2.391150 
 -2.391150 
 -2.497491 
 -7.275712 
 -7.275712 
 -5.923299 
 -6.710426 
 2013 
 213 
 3 
 
 
 4 
 0.352317 
 0.509844 
 0.188443 
 0.143825 
 0.089402 
 0.111278 
 0.307617 
 0.312690 
 0.389198 
 0.331207 
 2.146012 
 3.417696 
 1.553117 
 3.497509 
 3.497509 
 3.488768 
 0.528852 
 0.528852 
 2.050237 
 2.396472 
 -4.407169 
 -3.920889 
 -4.087314 
 -1.891085 
 -1.891085 
 -1.964027 
 -5.059037 
 -5.059037 
 -4.223137 
 -5.095089 
 3.158082 
 4.636039 
 2.179199 
 4.430997 
 4.430997 
 4.420446 
 0.868211 
 0.868211 
 3.020928 
 3.220925 
 -6.276412 
 -5.191124 
 -5.500988 
 -2.484262 
 -2.484262 
 -2.632834 
 -7.140369 
 -7.140369 
 -5.934493 
 -6.825925 
 2013 
 213 
 4


Solution data frame saved to data/Solution_pp_2018_10_21_00_41_10.pkl

Dataframe solution_df for feather:



,TARGETVAR_1,TARGETVAR_2,TARGETVAR_3,TARGETVAR_4,TARGETVAR_5,TARGETVAR_6,TARGETVAR_7,TARGETVAR_8,TARGETVAR_9,TARGETVAR_10,U10_1,U10_2,U10_3,U10_4,U10_5,U10_6,U10_7,U10_8,U10_9,U10_10,V10_1,V10_2,V10_3,V10_4,V10_5,V10_6,V10_7,V10_8,V10_9,V10_10,U100_1,U100_2,U100_3,U100_4,U100_5,U100_6,U100_7,U100_8,U100_9,U100_10,V100_1,V100_2,V100_3,V100_4,V100_5,V100_6,V100_7,V100_8,V100_9,V100_10,YEAR,DAYOFYEAR,HOUR
0,0.000000,0.270840,0.070839,0.032609,0.029816,0.052044,0.140550,0.224602,0.054787,0.121128,0.566472,1.552273,-0.056684,0.691553,0.691553,1.167131,-0.465222,-0.465222,0.781539,-0.165117,-3.866858,-3.037934,-3.197154,-1.882241,-1.882241,-2.284945,-4.393509,-4.393509,-3.757118,-2.658255,0.935726,2.819186,0.151737,1.262296,1.262296,1.829496,-0.484770,-0.484770,1.473541,0.320843,-5.888026,-5.077052,-4.559475,-2.754384,-2.754384,-3.287274,-6.763927,-6.763927,-5.748985,-4.277716,2013,213,0
1,0.019168,0.195737,0.081343,0.028040,0.041659,0.051252,0.152884,0.150140,0.184262,0.235245,2.513531,1.856405,1.865527,2.099645,2.099645,2.254329,-0.144996,-0.144996,2.461843,-0.349089,-4.475081,-3.909570,-3.798061,-1.608958,-1.608958,-1.636207,-5.039754,-5.039754,-4.089410,-3.789677,3.426426,2.678273,2.653962,2.709924,2.709924,2.918636,-0.033150,-0.033150,3.378262,-0.350576,-6.415826,-5.350381,-5.098012,-2.039563,-2.039563,-2.081795,-7.279274,-7.279274,-5.847995,-5.323414,2013,213,1
2,0.286627,0.207469,0.107185,0.064430,0.052353,0.086763,0.199029,0.151965,0.389327,0.271003,2.730166,2.283594,1.522712,2.638951,2.638951,2.653013,0.056817,0.056817,2.637051,0.769431,-4.752177,-4.232779,-3.935561,-1.645848,-1.645848,-1.640399,-5.192765,-5.192765,-4.283922,-4.633122,3.915867,3.169549,2.140667,3.317253,3.317253,3.320464,0.207672,0.207672,3.845685,1.020502,-6.847804,-5.587994,-5.242004,-2.112832,-2.112832,-2.149975,-7.343893,-7.343893,-6.058642,-6.176686,2013,213,2
3,0.287943,0.395594,0.157022,0.055293,0.052712,0.103719,0.281050,0.217666,0.503941,0.272300,2.461083,3.095404,1.449740,2.958202,2.958202,2.892832,0.359725,0.359725,2.187819,1.873888,-4.583656,-4.070965,-4.013953,-1.850422,-1.850422,-1.892762,-5.167193,-5.167193,-4.210980,-5.047719,3.541562,4.191092,2.054430,3.704862,3.704862,3.611744,0.637481,0.637481,3.203953,2.476900,-6.545063,-5.364627,-5.389050,-2.391150,-2.391150,-2.497491,-7.275712,-7.275712,-5.923299,-6.710426,2013,213,3
4,0.352317,0.509844,0.188443,0.143825,0.089402,0.111278,0.307617,0.312690,0.389198,0.331207,2.146012,3.417696,1.553117,3.497509,3.497509,3.488768,0.528852,0.528852,2.050237,2.396472,-4.407169,-3.920889,-4.087314,-1.891085,-1.891085,-1.964027,-5.059037,-5.059037,-4.223137,-5.095089,3.158082,4.636039,2.179199,4.430997,4.430997,4.420446,0.868211,0.868211,3.020928,3.220925,-6.276412,-5.191124,-5.500988,-2.484262,-2.484262,-2.632834,-7.140369,-7.140369,-5.934493,-6.825925,2013,213,4



Solution data frame saved to data/Solution_pp_2018_10_21_00_41_10.feather
